In [1]:
import pandas  as pd
from datetime import datetime, timedelta
import pytz
import time
import json

from openpyxl import Workbook, load_workbook
from openpyxl.styles import Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime
from dateutil.relativedelta import relativedelta
# Create plotly visualization
import plotly.graph_objects as go

import epint as ep
print(ep.__version__)

In [2]:
ep.set_auth("ahmetfaruk.kornaz@yepas.com", "Kpyrba2321!")

# Verilen uevcb_list ID'lerine göre santral bilgilerini filtrele
uevcb_ids = [
    3197267,
    3205381,
    3205710,
    134405,
    3205527,
    3204758,
    3204759,
    1740316,
    924,
    928,
    3204385,
    3204399,
    3204400,
    24604,
    3194367,
    3205524,
    3205525,
    2543,
    945,
    983,
    3195727,
    3206732,
    3206733,
    301420,
    472111,
]

In [3]:
import pytz

# Europe/Istanbul timezone'unu tanımla
istanbul_tz = pytz.timezone('Europe/Istanbul')

now = datetime.now(istanbul_tz)

# Eğer saat 13:30'dan büyükse end yarın olsun, değilse bugün olsun
if now.hour > 13 or (now.hour == 13 and now.minute >= 30):
    end_date = now + timedelta(days=1)
else:
    end_date = now

end_date = datetime(end_date.year, end_date.month, end_date.day+1)

# Start tarihi end tarihinden 15 gün öncesi olsun
start_date = end_date - timedelta(days=2)

print(f"Start Date: {start_date}")
print(f"End Date: {end_date}")

In [4]:
def split_date_by_day(date_start, date_end):
    """
    Verilen tarih aralığını günlük periyotlara böler.
    Her periyot bir günü temsil eder.
    start: mevcut gün, end: bir sonraki gün
    """
    periyotlar = []
    
    # Başlangıç tarihini normalize et
    current_start = date_start
    
    while current_start < date_end:
        # Her gün için bir periyot oluştur
        # start: mevcut gün, end: bir sonraki gün
        next_day = current_start + relativedelta(days=1)
        
        periyotlar.append({
            'start': current_start,
            'end': next_day
        })
        
        # Bir sonraki güne geç
        current_start = next_day
    
    return periyotlar

In [5]:
def fetch_service(date_start, date_end, service, **params):
    # Günlük periyotlara böl (split_date_by_day kullan)
    periyotlar = split_date_by_day(date_start, date_end)
    
    # Tüm günlerin verilerini birleştirmek için liste
    all_data = []

    for p in periyotlar:
        # Her gün için aynı tarihi start ve end olarak kullan
        data = service(
            **params, 
            startdate = p['start'].strftime('%Y-%m-%d'),
            endtdate = p['start'].strftime('%Y-%m-%d'),
        )
                    
        # print(data)
        if data and 'items' in data and data['items']:
            df_temp = pd.DataFrame(data['items'])
            all_data.append(df_temp)
        
        # API rate limit için bekleme
        time.sleep(0.8)

    # Tüm günlerin verilerini birleştir
    if all_data:
        df = pd.concat(all_data, ignore_index=True)
    else:
        df = pd.DataFrame()
    
    return df

In [6]:
org_list = fetch_service(
    start_date, 
    end_date, 
    ep.seffaflik_electricity.organization_list, 
    # uevcbIds = uevcb_list
)

uevcb_list = fetch_service(
    start_date, 
    end_date, 
    ep.seffaflik_electricity.uevcb_list_bulk, 
    organizationIds = list(org_list['organizationId'].unique())
)

uevcb_list = uevcb_list.drop_duplicates()

# uevcb_list

In [7]:
# uevcb_list DataFrame'inden bu ID'lere sahip santralları filtrele
filtered_santraller = uevcb_list[uevcb_list['id'].isin(uevcb_ids)]

# Dictionary oluştur: {uevcb_id: {'orgId': orgId, 'name': name}}
santral_dict = {}
for _, row in filtered_santraller.iterrows():
    santral_dict[row['id']] = {
        'orgId': row['orgId'],
        'name': row['name']
    }

In [8]:
# Santral isimlerini değiştir - dict haline getir
santral_kisa_uzun = {
    "ERZIN": "Erzin Santrali",
    "ALIAGA_HABAS": "ALİAĞA DGKÇS(Habaş DGKÇS/DB-1)",
    "BURSA_DG_1": "BURSA DG BLOK1",
    "BURSA_DG_2": "BURSA DG BLOK2",
    "TEKIRDAG_B": "TEKİRDAĞ DGKÇS_B(Doğalgaz/Motorin)",
    "TEKIRDAG_A": "TEKİRDAĞ DGKÇS_A(Doğalgaz/Motorin)",
    "BANDIRMA_ENJ": "ENERJİSA BANDIRMA SANTRALI",
    "BANDIRMA_2": "BANDIRMA II DGKÇS",
    "SAMSUN_P1": "BİLGİN SAMSUN DGKÇS PT1(Doğalgaz)",
    "SAMSUN_P2": "BİLGİN SAMSUN DGKÇS PT2(Doğalgaz)",
    "HAMITABAT_10": "HAMİTABAT ÜNİTE-10",
    "HAMITABAT_20": "HAMİTABAT ÜNİTE-20",
    "ANTALYA_DG": "ANTALYA ENERJİ SANTRALİ(Doğalgaz)",
    "CENGIZ_610": "CENGİZ 610 DGKÇS",
    "RWE_GUNEY": "RWE_TURCAS_GUNEY",
    "YENI_KCC": "Yeni Elektrik Doğalgaz Kombine Çevrim Santrali",
    "IC_ANADOLU": "İÇ ANADOLU DGKÇS",
    "KIRIKKALE_ACWA": "ACWA POWER KIRIKKALE DGKÇS",
    "ADAPAZARI": "ADAPAZARI DGKÇS",
    "GEBZE_1": "GEBZE DGKÇS_1",
    "GEBZE_2": "GEBZE DGKÇS_-2",
    "ANKARA": "ANKARA DGKÇS",
    "IZMIR_GR1": "İZMİR DGKÇS_GR1(Doğalgaz)",
    "IZMIR_GR2": "İZMİR DGKÇS_GR2(Doğalgaz)",
}

santral_short_names_for_input = list(santral_kisa_uzun.keys())

### kgup son \(kudup\)

In [9]:
kgup_new = pd.DataFrame()
for uevcb_id, pp_info in santral_dict.items():
    # print(pp_info)
    data = fetch_service(
        start_date, 
        end_date, 
        ep.seffaflik_electricity.sbfgp, 
        organizationId = pp_info['orgId'],
        uevcbId = uevcb_id
        )
    
    data['uevcbId'] = uevcb_id
    data['uevcbName'] = pp_info['name']
    data['orgId'] = pp_info['orgId']
    kgup_new = pd.concat([kgup_new, data], ignore_index=True)

### kgup first

In [10]:
kgup_first_new = pd.DataFrame()
for uevcb_id, pp_info in santral_dict.items():
    # print(pp_info)
    data = fetch_service(
        start_date, 
        end_date, 
        ep.seffaflik_electricity.dpp_first_version, 
        organizationId = pp_info['orgId'],
        uevcbId = uevcb_id
        )
    
    data['uevcbId'] = uevcb_id
    data['uevcbName'] = pp_info['name']
    data['orgId'] = pp_info['orgId']
    kgup_first_new = pd.concat([kgup_first_new, data], ignore_index=True)

### aic \(eak\)

In [12]:
aic_new = pd.DataFrame()
for uevcb_id, pp_info in santral_dict.items():
    # print(pp_info)
    data = fetch_service(
        start_date, 
        end_date, 
        ep.seffaflik_electricity.aic, 
        organizationId = pp_info['orgId'],
        uevcbId = uevcb_id
        )
    
    data['uevcbId'] = uevcb_id
    data['orgId'] = pp_info['orgId']
    aic_new = pd.concat([aic_new, data], ignore_index=True)
# aic_new

In [13]:
# aic DataFrame'ine santral isimlerini ekle
# uevcb_id'ye göre santral_dict'ten name bilgisini al
aic_new['name'] = aic_new['uevcbId'].map(lambda x: santral_dict.get(x, {}).get('name', None))

aic_new

In [14]:
# Remove timezone from aic_new date column
if 'date' in aic_new.columns:
    aic_new['date'] = pd.to_datetime(aic_new['date']).dt.tz_localize(None)

# Remove timezone from kgup_first_new date column
if 'date' in kgup_first_new.columns:
    kgup_first_new['date'] = pd.to_datetime(kgup_first_new['date']).dt.tz_localize(None)

# Remove timezone from aic_new date column
if 'date' in kgup_new.columns:
    kgup_new['date'] = pd.to_datetime(kgup_new['date']).dt.tz_localize(None)


aic_new = aic_new[['date', 'time', 'toplam', 'dogalgaz', 'orgId', 'uevcbId', 'name']]
kgup_new = kgup_new[['date', 'time', 'toplam', 'dogalgaz', 'orgId', 'uevcbId', 'uevcbName']]

In [15]:
# Read the parquet file
aic = pd.read_parquet('/work/inputs/dogalgaz/aic.parquet')
kgup = pd.read_parquet('/work/inputs/dogalgaz/kgup.parquet')
kgup_first = pd.read_parquet('/work/inputs/dogalgaz/kgup_first.parquet')

In [18]:
aic = aic_new.combine_first(aic)


# Update kgup_first with kgup_first_new
kgup_first = kgup_first_new.combine_first(kgup_first)

# For kgup, we need special handling:
# If kgup_new has NaN values for certain date/time/uevcbId combinations,
# those same combinations in kgup should also be set to NaN

# First, update kgup with kgup_new values
kgup = kgup_new.combine_first(kgup)


> kgup dataframeini kgup\_new ile update etmek istiyorum fakat soyle bir durum olucak cakisan tarihleri olucak bunlar zaten kgup\_new ile guncellnice cakismayan ayri tarihleri var bu da kgup\_new den glesin istiyorum ayrica eger deger nan ise de nan olarak gecsin kgup\_new override kgup entirely

\`kgup\_new\`'in \`kgup\`'ı tamamen override etmesi için şu kodu kullanabiliriz:

In [19]:
# aic_new'in aic'ı tamamen override etmesi için:
# 1. Önce aic_new ve aic'ı birleştireceğimiz key'leri belirleyelim
# 2. aic'tan aic_new'de olan kayıtları çıkaralım
# 3. Sonra aic_new ile birleştirelim

# Merge key'leri: date, time, uevcbId
merge_keys = ['date', 'time', 'uevcbId']

# aic'tan, aic_new'de olan (date, time, uevcbId) kombinasyonlarını çıkar
aic_filtered = aic[~aic.set_index(merge_keys).index.isin(aic_new.set_index(merge_keys).index)]
aic = pd.concat([aic_filtered, aic_new], ignore_index=True)
aic = aic.sort_values(['date', 'time', 'uevcbId']).reset_index(drop=True)

# kgup'tan, kgup_new'de olan (date, time, uevcbId) kombinasyonlarını çıkar
kgup_filtered = kgup[~kgup.set_index(merge_keys).index.isin(kgup_new.set_index(merge_keys).index)]
kgup = pd.concat([kgup_filtered, kgup_new], ignore_index=True)
kgup = kgup.sort_values(['date', 'time', 'uevcbId']).reset_index(drop=True)

# kgup_first'tan, kgup_first_new'de olan (date, time, uevcbId) kombinasyonlarını çıkar
kgup_first_filtered = kgup_first[~kgup_first.set_index(merge_keys).index.isin(kgup_first_new.set_index(merge_keys).index)]
kgup_first = pd.concat([kgup_first_filtered, kgup_first_new], ignore_index=True)
kgup_first = kgup_first.sort_values(['date', 'time', 'uevcbId']).reset_index(drop=True)

kgup dataframe'i başarıyla güncellendi\. kgup\_new'deki tüm veriler \(NaN değerler dahil\) kgup'ı tamamen override etti\. Çakışan tarih/saat/uevcbId kombinasyonları kgup\_new'den geldi, çakışmayan eski tarihler ise kgup'ta kaldı\.

In [20]:
# Save kgup to parquet
kgup.to_parquet('/work/inputs/dogalgaz/kgup.parquet', index=False)

kgup_first.to_parquet('/work/inputs/dogalgaz/kgup_first.parquet', index=False)

aic.to_parquet('/work/inputs/dogalgaz/aic.parquet', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3162b760-2d3a-4bdb-88e8-2b0bee1ebdd1' target="_blank">

Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>